
# vbt-extensions — Demo Completa (BTCUSDT)

Este notebook demonstra um fluxo **end-to-end** usando `vbt_extensions`:

1. **Dados** (Binance OHLCV) — com fallback sintético se indisponível  
2. **Indicador** (Trend Lines)  
3. **Sinais** (Breakout de linha de tendência)  
4. **Backtest** (vectorbt `Portfolio.from_signals`)  
5. **Relatórios e Plots**



> **Pré-requisitos**  
> - `vectorbt`, `pandas`, `numpy`  
> - `vbt_extensions` instalado a partir do seu repositório  
> - (Opcional) `python-binance` e credenciais, se quiser baixar dados reais


In [1]:
import sys

sys.path.insert(0, "..")

In [2]:
import pandas as pd
import numpy as np
import vectorbt as vbt

# exibir versões para debug
print("vectorbt:", vbt.__version__)

vectorbt: 0.28.1


In [ ]:
import numpy as np
import pandas as pd
import vectorbt as vbt
from binance.client import Client

from vbt_extensions.data.binance import BinanceDownloadParams, binance_download

client = Client()
params = BinanceDownloadParams(
    client=client, symbol="BTCUSDT",
    interval="1h", start="90 days ago UTC", tz="UTC",
    fill_gaps=True, drop_partial_last=True,
)
df = binance_download(params)
df.head()

0it [00:00, ?it/s]

c:\Users\gabriel\workspace\vbt_extensions\examples\..\vbt_extensions\data\base.py:107: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.

c:\Users\gabriel\workspace\vbt_extensions\examples\..\vbt_extensions\data\base.py:127: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



## 1) Dados — Binance (com fallback sintético)

In [1]:

USE_SYNTH = False
df = None

try:
    from binance.client import Client
    client = Client()  # usa credenciais do ambiente se configuradas

    params = BinanceDownloadParams(
        client=client, ticker="BTCUSDT",
        interval="1h", start="90 days ago UTC", tz="UTC",
        fill_gaps=True, drop_partial_last=True
    )
    df = binance_download(params)
    print("Dados Binance carregados:", df.shape, df.index.min(), "->", df.index.max())
except Exception as e:
    print("Falha ao baixar Binance, usando série sintética. Motivo:", e)
    USE_SYNTH = True

if USE_SYNTH:
    # Série sintética via GBM para a demo ficar auto-contida
    idx = pd.date_range(end=pd.Timestamp.utcnow().floor("H"), periods=24*90, freq="H", tz="UTC")
    np.random.seed(42)
    mu, sigma = 0.0002, 0.02
    returns = np.random.normal(mu, sigma, size=len(idx))
    close = 30000 * np.exp(np.cumsum(returns))
    high = close * (1 + np.random.uniform(0, 0.01, len(idx)))
    low = close * (1 - np.random.uniform(0, 0.01, len(idx)))
    open_ = pd.Series(close).shift(1).fillna(close[0]).values
    vol = np.random.uniform(10, 100, len(idx))
    df = pd.DataFrame({
        "Open": open_,
        "High": high,
        "Low": low,
        "Close": close,
        "Volume": vol
    }, index=idx).astype(float)
    print("Dados sintéticos gerados:", df.shape, df.index.min(), "->", df.index.max())

df.tail()


Falha ao baixar Binance, usando série sintética. Motivo: name 'BinanceDownloadParams' is not defined


NameError: name 'pd' is not defined

## 2) Indicador — Trend Lines

In [ ]:

tl = TREND_LINE.run(
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    lookback=100,
    touch_tol=0.001,
    compute_events=True,
)
tl.support_level.tail(), tl.resist_level.tail()


## 3) Sinais — Breakout nas linhas de tendência

In [ ]:

entries, exits, _, _ = tl_breakout_sig(
    df["Close"],
    resist_level=tl.resist_level,
    support_level=tl.support_level,
    confirm_shift=1
)
entries.tail(), exits.tail()


### (Opcional) Regime por slope

In [ ]:

regime = classify_trend_regime(
    support_slope=tl.support_slope,
    resist_slope=tl.resist_slope,
    up_thresh=0.0001,
    down_thresh=0.0001,
)
regime.value_counts()


## 4) Backtest — Portfolio.from_signals

In [ ]:

pf = vbt.Portfolio.from_signals(
    close=df["Close"],
    entries=entries,
    exits=exits,
    slippage=0.0005, fees=0.0005, init_cash=100_000,
    freq="H"
)
pf.stats()


## 5) Plots — Preço + Trend Lines + Regime

In [ ]:

fig = plot_trend_lines(
    df["Close"],
    support_level=tl.support_level, resist_level=tl.resist_level,
    touch_support=tl.touch_support, touch_resist=tl.touch_resist,
    break_up=tl.break_up, break_down=tl.break_down,
    title="Trend Lines + Eventos"
)
fig = plot_regime_background(df["Close"], regime, title="Trend Lines + Regime", fig=fig)
fig.show()


## 6) Relatório — full_tearsheet

In [ ]:

full_tearsheet(pf)
